<a href="https://colab.research.google.com/github/leeCodingStudio/Python_NLP/blob/master/11_BERT%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_%EB%8B%A8%EC%96%B4_%EC%B6%94%EB%A1%A0_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 허깅페이스
* 트랜스포머를 기반으로 다양한 모델과 학습 데이터, 학습 방법을 구현해 놓은 모듈
* 질의응답, 텍스트 분류, 텍스트 요약, 개체명 인식, 텍스트 생성, 번역, 언어 모델

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.5 MB/s eta 0:00:00


In [2]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [3]:
# KLUE
# 한국어 자연어처리 과제에 사용되는 언어 모델.
# 카카오에서 개발한 한국어 BERT
# 사전 훈련(pre-training)과 다양한 작업에 대해 세부 조정(fine-tuning)하는 과정을 통해 학습
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [4]:
text = '[CLS] 이순신은 누구입니까? [SEP] 16세기 말 조선의 명장이자 충무공이며 임진왜란 및 정유재란 당시 조선 수군을 지휘했던 제독이다 [SEP]'
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

['[CLS]', '이순신', '##은', '누구', '##입', '##니까', '?', '[SEP]', '16', '##세기', '말', '조선', '##의', '명장', '##이', '##자', '충무', '##공', '##이', '##며', '임진왜란', '및', '정유', '##재', '##란', '당시', '조선', '수군', '##을', '지휘', '##했', '##던', '제독', '##이다', '[SEP]']


In [5]:
masked_index = 16
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

['[CLS]', '이순신', '##은', '누구', '##입', '##니까', '?', '[SEP]', '16', '##세기', '말', '조선', '##의', '명장', '##이', '##자', '[MASK]', '##공', '##이', '##며', '임진왜란', '및', '정유', '##재', '##란', '당시', '조선', '수군', '##을', '지휘', '##했', '##던', '제독', '##이다', '[SEP]']


In [6]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

[2, 10661, 2073, 4061, 2372, 3707, 35, 3, 3879, 14223, 1041, 3957, 2079, 17449, 2052, 2155, 4, 2086, 2052, 2307, 15294, 1116, 9530, 2070, 2241, 3817, 3957, 15560, 2069, 5872, 2371, 2414, 28348, 28674, 3]


In [7]:
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [8]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [9]:
model = BertModel.from_pretrained('klue/bert-base')
model.eval()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [11]:
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [13]:
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    encoded_layers = outputs[0]

print(encoded_layers)
print(encoded_layers.shape)

tensor([[[ 1.0652, -0.5908, -1.5817,  ...,  0.7246, -0.0901,  1.0030],
         [ 0.8961,  0.3418, -0.7090,  ...,  0.8243,  0.2939,  1.3385],
         [-0.7230, -0.8876, -1.0106,  ...,  2.3141, -0.2984,  0.1895],
         ...,
         [-0.0042, -0.0462, -0.2227,  ...,  1.1374,  0.7977,  1.7137],
         [-0.5353, -0.9521, -1.7591,  ...,  1.3387,  0.5158,  1.6115],
         [-0.0802, -0.9287, -1.7088,  ...,  0.1650, -0.5035,  1.4360]]],
       device='cuda:0')
torch.Size([1, 35, 768])


In [17]:
predicted_index = torch.argmax(encoded_layers[0, masked_index]).item()
print(predicted_index)
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

563
갱


In [20]:
model = BertForMaskedLM.from_pretrained('klue/bert-base')
model.eval()

tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    encoded_layers = outputs[0]

predicted_index = torch.argmax(encoded_layers[0, masked_index]).item()
print(predicted_index)
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


15100
충무
